In [2]:
import numpy as np

In [10]:
def optimal_elimination_literal(a: np.ndarray, b: np.ndarray, show_annotation: bool = False) -> np.ndarray:
    
    if a.shape[0] != a.shape[1]:
        raise ValueError("Matrix a must be square")
    
    if a.shape[0] != b.shape[0]:
        raise ValueError("Dimensions of a and b must match")
    
    det_a = np.linalg.det(a)
    if abs(det_a) < 1e-12:
        raise ValueError("Determinant is near zero — no unique solution")
    
    n = len(b)
    
    E = np.zeros((n, n + 1), dtype=float)
    E[:, :n] = a.astype(float)
    E[:, n] = b.astype(float)

    cols = np.arange(n, dtype=int)

    if show_annotation:
        print("System:\n", E, "\ncols:", cols, "\n")

    for k in range(n):
        # максимальный элемент в строке, с последующей перестановкой строк
        p = k + int(np.argmax(np.abs(E[k, k:n])))

        if abs(E[k, p]) < 1e-12:
            raise ValueError(f"Zero leading element at step {k}, column {p}")

        if p != k:
            E[:, [k, p]] = E[:, [p, k]]
            cols[[k, p]] = cols[[p, k]]
            if show_annotation:
                print(f"Step {k+1}: swapped columns {k} and {p}")

        E_next = E.copy()

        if k == 0:
            denom = E[0, 0]
        else:
            denom = E[k, k] - sum(E[r, k] * E[k, r] for r in range(0, k))

        if abs(denom) <1e-12:
            raise ValueError(f"Denominator near zero at step {k}")

        # k-я строка для столбцов p = k..n
        for pcol in range(0, n + 1):
            numer = E[k, pcol]
            if k > 0:
                numer -= sum(E[r, pcol] * E[k, r] for r in range(0, k))
            E_next[k, pcol] = numer / denom

        # теперь для i = 0..k-1 и p = k+1..n
        for i in range(0, k):
            for pcol in range(k + 1, n + 1):
                E_next[i, pcol] = E[i, pcol] - E_next[k, pcol] * E[i, k]
            E_next[i, k] = 0

        E_next[k, k] = 1

        E = E_next

        if show_annotation:
            print(f"After step {k+1}:")
            print(E)
            print("cols:", cols, "\n")

    x_perm = E[:, n].copy()

    ans = np.zeros(n, dtype=float)
    for j in range(n):
        ans[cols[j]] = x_perm[j]

    return ans

In [11]:
a = np.array([[2, 3, 2], [2, 3, 4], [3, 4, 5]])
b = np.array([7, 9, 12])
optimal_elimination_literal(a, b, show_annotation=True)

System:
 [[ 2.  3.  2.  7.]
 [ 2.  3.  4.  9.]
 [ 3.  4.  5. 12.]] 
cols: [0 1 2] 

Step 1: swapped columns 0 and 1
After step 1:
[[ 1.          0.66666667  0.66666667  2.33333333]
 [ 3.          2.          4.          9.        ]
 [ 4.          3.          5.         12.        ]]
cols: [1 0 2] 

Step 2: swapped columns 1 and 2
After step 2:
[[ 1.          0.          0.66666667  1.66666667]
 [ 0.          1.          0.          1.        ]
 [ 4.          5.          3.         12.        ]]
cols: [1 2 0] 

After step 3:
[[1. 0. 0. 1.]
 [0. 1. 0. 1.]
 [0. 0. 1. 1.]]
cols: [1 2 0] 



array([1., 1., 1.])